In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory

In [10]:
# Specify which GPU to use (e.g., GPU 0)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_memory_growth(gpus[0], True)

In [11]:
tf.config.experimental.list_physical_devices('GPU')

[]

In [2]:
# Define the batch size and image dimensions
batch_size = 64
img_height = 224
img_width = 224

# Define the path to your dataset folder
training_data_dir = "../data/chest_xray/train/"
val_data_dir = "../data/chest_xray/val"
test_data_dir = "../data/chest_xray/test/"

In [3]:
params = {
    "labels":"inferred",  # Automatically infer class labels from subdirectories
    "label_mode":"categorical",  # Use categorical labels
    "class_names":None,  # Automatically get class names from subdirectories
    "color_mode":"rgb",  # Use 'rgb' for color images or 'grayscale' for grayscale
    "batch_size":batch_size,
    "image_size":(img_height, img_width),
    "shuffle":True,  # Shuffle the dataset
    "seed":123,  # Set a seed for reproducibility
}

In [4]:
# Create a TensorFlow dataset from the directory
training_dataset = image_dataset_from_directory(
    training_data_dir,
    **params,
)
val_dataset = image_dataset_from_directory(
    val_data_dir,
    **params,
)
test_dataset = image_dataset_from_directory(
    test_data_dir,
    **params,
)

Found 5216 files belonging to 2 classes.
Found 16 files belonging to 2 classes.
Found 624 files belonging to 2 classes.


In [17]:
# Print the class names
print(training_dataset.class_names)

# Print the dataset information
print(training_dataset.element_spec)

['NORMAL', 'PNEUMONIA']
(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))


In [18]:
class BasicCNN(tf.keras.Model):
    def __init__(self):
        super(BasicCNN, self).__init__()
        self.conv_block_1 = tf.keras.Sequential([
            Conv2D(64, (3, 3), padding='same', strides=1, input_shape=(None, 3, 64, 64)),
            BatchNormalization(),
            ReLU()
        ])
        self.conv_block_2 = tf.keras.Sequential([
            Conv2D(128, (3, 3), padding='same', strides=1),
            BatchNormalization(),
            ReLU()
        ])
        self.conv_block_3 = tf.keras.Sequential([
            Conv2D(128, (3, 3), padding='same', strides=1),
            BatchNormalization(),
            ReLU()
        ])

        self.pool = MaxPooling2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.fc = Dense(1)
        self.sigmoid = sigmoid

    def call(self, x):
        x = self.pool(self.conv_block_1(x))
        x = self.pool(self.conv_block_2(x))
        x = self.pool(self.conv_block_3(x))
        x = self.flatten(x)
        x = self.sigmoid(self.fc(x))
        return x